In [1]:
import numpy as np


In [2]:
import csv
# 필드 목록 정의
# 비정형 필드
title_arr =[]
# 분류값을 저장할 배열 (0, 1)
ctr_arr =[]
# 텍스트와 분류값이 저장된 파일 불러오기(학습용 데이터셋)
f = open('c:/data/text/ratings_train.csv', encoding='ms949')
reader = csv.reader(f)
max_length=0
length=0
max_idx=0
count=0
next(reader,None) # 헤더 스킵(첫라인을 건너뜀)
count_train=0
for line in reader:
    # print(line)
    # 레코드 전체를 하는 방법
    # 텍스트 필드
    title_arr.append(line[0])
    # 결과 필드
    ctr_arr.append(int(line[1]))
    # 텍스트의 길이
    length=len(line[0])
    # print(length)
    if max_length < length :
        max_length=length
        max_idx=count
    count += 1
    count_train += 1
    if count_train>=10000: break
f.close()
# 검증용 데이터셋 파일 불러오기
f2 = open('c:/data/text/ratings_test.csv', 'r', encoding='ms949')
reader = csv.reader(f2)
# 헤더 스킵 (첫라인을 건너뜀)
next(reader,None)
count_test=0
for line in reader:
    # print(line)
    # 레코드 전체를 하는 방법
    title_arr.append(line[0]) # 텍스트 필드
    # 결과 필드
    ctr_arr.append(int(line[1]))
    # 텍스트의 길이
    length=len(line[0])
    # print(length)
    if max_length < length :
        max_length=length
        max_idx=count
    count += 1
    count_test += 1
    if count_test>=10000: break
f2.close()
print("count_train:",count_train)
print("count_test:",count_test)
print("title_arr size:", len(title_arr))
print("ctr_arr size:", len(ctr_arr))
print("max length:", max_length)

FileNotFoundError: ignored

In [ ]:
from konlpy.tag import Okt
okt = Okt()
# 명사(Noun), 형용사(Adjective), 동사(Verb), 알파벳(Alpha), 부사(Adverb)
title_noun_arr = []
for index, title in enumerate(title_arr):
    if index % 1000 == 0:
        print('step:',index)
    #명사만 추출
    title_noun_arr.append(okt.nouns(title))
print(len(title_noun_arr))

step: 0
step: 1000
step: 2000
step: 3000
step: 4000
step: 5000
step: 6000
step: 7000
step: 8000
step: 9000
step: 10000
step: 11000
step: 12000
step: 13000
step: 14000
step: 15000
step: 16000
step: 17000
step: 18000
step: 19000
20000


In [ ]:
# 벡터모델을 불러와서 텍스트를 벡터로 변환
import gensim
MIN_NOUN_VECTOR_VALUE = -10.0
MAX_NOUN_VECTOR_VALUE = 10.0
NOUN_VECTOR_SIZE = 100
# 벡터 생성 함수
def generate_random_noun_vector():
    return np.random.uniform(low=MIN_NOUN_VECTOR_VALUE,
    high=MAX_NOUN_VECTOR_VALUE,
    size=(NOUN_VECTOR_SIZE,))
# word2vec 모델 불러오기
w2v_model = gensim.models.Word2Vec.load('c:/data/text/text_100.model')
title_noun_vector_arr = [] # 벡터를 저장할 리스트
for index, title_nouns in enumerate(title_noun_arr):
    if index % 10000 == 0:
        print('step:',index)
    noun_vector_arr = []
    for noun in title_nouns:
        try:
            # 단어별로 벡터모델 적용
            noun_vector = w2v_model[noun]
        except Exception as e:
            # 단어사전에 존재하지 않으면 기본 벡터 생성
            noun_vector = generate_random_noun_vector()
        noun_vector_arr.append(noun_vector)
    # if len(noun_vector_arr)>0:
    # print(len(noun_vector_arr))
    title_noun_vector_arr.append(noun_vector_arr)

step: 0
step: 10000


In [ ]:
print(title_noun_vector_arr[:1])

[[array([ 2.00674496,  9.92537925,  2.4558384 , -1.59299407,  3.22687172,
        9.96287205, -5.72706165,  9.33376551, -0.89992089, -2.47658237,
        8.6599801 , -3.07644587, -4.90290539, -9.78793655, -1.18713767,
        0.35956016,  1.67120584, -3.32069043,  0.60090417,  7.77693439,
       -6.11084429,  3.19661383, -7.86466089,  5.94472287,  4.42777076,
       -8.14705539, -2.30804139,  7.97123681, -7.4176351 ,  9.18937291,
       -0.89086534,  7.70903556, -3.90824298,  4.60130148,  4.04569175,
       -1.66317635, -2.10432484,  8.79874805,  2.31572135, -8.5166547 ,
       -5.16584091, -1.9202235 ,  3.27869836, -8.98360174,  3.96186372,
        2.31275087, -0.113988  , -0.84982274, -2.38649969, -5.98094787,
        9.19409368,  4.37595363, -6.01333228,  1.65173564,  1.89329101,
       -4.35323374, -6.30220786, -9.10938411, -3.62772455, -8.77632888,
        6.25053505, -7.32394066,  6.29351339, -5.09580588, -6.95011574,
       -4.65491375, -8.4097219 ,  0.78954514,  2.65206253,  3.

In [ ]:
max_size=0 # 가장 긴 텍스트 사이즈를 저장할 변수
min_size=100
avg=0
count=0
sum_size=0
# 가장 긴 리뷰 텍스트 계산
for index, title_noun_vector in enumerate(title_noun_vector_arr):
    # print(len(title_noun_vector))
    if len(title_noun_vector)==0:
        pass
        # print(title_noun_vector)
    sum_size += len(title_noun_vector)
    if max_size < len(title_noun_vector):
        max_size=len(title_noun_vector)
    if min_size > len(title_noun_vector):
        min_size=len(title_noun_vector)
    count += 1
    # print(":title_noun_vector len=>",len(title_noun_vector))
# padding
TITLE_LENGTH = max_size
def generate_zero_noun_vector():
    return np.random.uniform(low=0.0, high=0.0, size=(NOUN_VECTOR_SIZE,))

In [ ]:
# 인덱스와 요소를 하나씩 전개하여 패딩 처리
title_noun_vector_arr2=[]
for index, title_noun_vector in enumerate(title_noun_vector_arr):
    if index % 1000 == 0:
     print('step:',index)
    while len(title_noun_vector) < TITLE_LENGTH:
        title_noun_vector.append(generate_zero_noun_vector())
    title_noun_vector = title_noun_vector[:TITLE_LENGTH]
    title_noun_vector_arr2.append(title_noun_vector)

step: 0
step: 1000
step: 2000
step: 3000
step: 4000
step: 5000
step: 6000
step: 7000
step: 8000
step: 9000
step: 10000
step: 11000
step: 12000
step: 13000
step: 14000
step: 15000
step: 16000
step: 17000
step: 18000
step: 19000


In [ ]:
print(min_size,max_size)

0 37


In [ ]:
# ctr to classification
# 클래스 개수
NUM_CLASSES = 2
ctr_class_arr = []
# 클래스 개수 저장
ctr_class_count = [0,0]
# print(success_arr)
for index, ctr in enumerate(ctr_arr):
    if ctr == 0:
        # ctr_class_arr.append([1,0])
        ctr_class_arr.append(0.0)
        ctr_class_count[0] += 1
    elif ctr == 1:
        # ctr_class_arr.append([0,1])
        ctr_class_arr.append(1.0)
        ctr_class_count[1] += 1
print(ctr_class_count)
print(ctr_class_arr[:10])

[9960, 10040]
[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0]


In [ ]:
# 데이터 전처리(0.0~1.0), 각 특성의 평균을 계산
def prepare_data(arr):
    sum1=0
    for i in arr:
        sum1 += i
    arr = [float(i)/sum1 for i in arr]
    return arr
# 전처리가 필요한 필드에 대해 함수 호출

In [ ]:
# 검증용 데이터셋의 개수 구하기
test_data_size = count_test
train_data_size=count_train
print(train_data_size,' ',test_data_size)
# 학습용 데이터셋과 검증용 데이터셋을 분리하는 코드
train_input = title_noun_vector_arr[0:train_data_size]
train_label = ctr_class_arr[0:train_data_size]
test_input = title_noun_vector_arr[train_data_size:]
test_label = ctr_class_arr[train_data_size:]
print("ctr_class_arr 개수:",len(ctr_class_arr)) 
print("train 총데이터개수:",len(train_input)) 
print("train 라벨개수:",len(train_label)) 
print("test 총데이터개수:",len(test_input)) 
print("test 라벨개수:",len(test_label)) 
print("train_data_size:",train_data_size)
print("test_data_size:",test_data_size)

10000   10000
ctr_class_arr 개수: 20000
train 총데이터개수: 10000
train 라벨개수: 10000
test 총데이터개수: 10000
test 라벨개수: 10000
train_data_size: 10000
test_data_size: 10000


In [ ]:
len(train_input)

10000

In [ ]:
len(test_input)

10000

In [ ]:
# 데이터를 케라스에 맞게 변환해주는 과정
# 데이터개수 가로 세로 채널수(흑백이미지의 경우 1, 컬러이미지의 경우 3, 여기서는 텍스트이므로 1을 사용)
# width, height, channel
train_input=np.array(train_input)
train_input = train_input.reshape(train_input.shape[0],
train_input.shape[1],NOUN_VECTOR_SIZE, 1)
test_input=np.array(test_input)
test_input = test_input.reshape(test_input.shape[0],
train_input.shape[1],NOUN_VECTOR_SIZE, 1)
# X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, Dense, Dropout, Flatten
from tensorflow.keras.models import Model
# 입력데이터
input_a = Input(shape=(train_input.shape[1], NOUN_VECTOR_SIZE, 1), name="input-layer")
# 첫번째 Conv 레이어
x=Conv2D(1, (3,3), activation='relu', padding="valid", strides=(1,1))(input_a)
# 두번째 Conv 레이어
x=Conv2D(1, (3,3), activation='relu')(x)
x=MaxPooling2D(pool_size=(2,2))(x)
x=Dropout(0.25)(x)
#fully connected network
x=Flatten()(x)
x=Dense(128, activation='relu')(x)
x=Dropout(0.5)(x)
# 출력계층 (1개의 클래스로 분류됨)
out=Dense(1, activation='sigmoid', name="output-layer")(x)
# CNN 모델이 생성됨
model = Model(inputs=[input_a], outputs=out)
# 모델 컴파일, 손실함수, 옵티마이저 설정
# CNN 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam',
metrics=['accuracy'])


In [ ]:
# 학습
from keras.callbacks import EarlyStopping
# 조기종료 콜백함수
early_stopping = EarlyStopping(patience = 2)
hist=model.fit( x=train_input, y=np.array(train_label),
validation_split=0.25, batch_size=256, epochs=5, verbose=1,
callbacks=[early_stopping])

In [ ]:
# 평가
scores = model.evaluate(np.array(test_input)[:100],
np.array(test_label)[:100], batch_size=1, verbose=1)
# 손실률
print("%s: %.2f%%" % (model.metrics_names[0], scores[0]*100))
# 정확도
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
# CNN의 최종 출력값이 필요할 경우 아래 코드 사용
m2 = Model(inputs=model.input, outputs=model.get_layer('output-layer').output)
y = m2.predict(test_input)
# y = m2.predict(train_input)

In [ ]:
y[0:10]

In [ ]:
model.save('c:/data/text')

In [ ]:
n=911
print(model.predict(np.array(train_input)[n].reshape(1, 37, 100, 1)))
print(np.array(train_label)[n])
print(title_arr[n])


In [ ]:
n=4992
print(model.predict(np.array(test_input)[n].reshape(1, 37, 100, 1)))
print(np.array(test_label)[n])
print(title_arr[n+10000])